For each binary cancer type or for multiclass (see settings below), creates a classifier with the given type and saves the models and results (in figs/)


In [1]:
# settings:

cancer_types = ['BLCA', 'BRCA', 'COAD', 'ESCA', 'HNSC', 'KIRC', 'KIRP', 'LIHC', 'LUAD', 'LUSC', 'PRAD', 'THCA', 'UCEC']
# cancer_types = [''] # uncomment to create a multiclass model
classifier_type = 'SVM' # can be LR (logistic regression) or SVM (support vector machine)

In [2]:
if cancer_types == ['']: # multiclass params
    params = {'penalty': 'l2',
                 'solver': 'lbfgs',
                 'max_iter': 100} # found from Hyperparam_search
else: # binary:  
    params = {'penalty': 'l2',
                 'solver': 'newton-cg',
                 'max_iter': 100}

if cancer_types == ['']: # multiclass
    params_SVM = {'kernel': 'rbf',
             'tol': 1e-3, 
             'gamma': 'auto',
             'probability': True} # found from Hyperparam_search for multiclass
else: # binary
    params_SVM = {'kernel': 'rbf',
             'tol': 1e-3, # default
             'gamma': 'scale', # default
             'probability': True} 

In [3]:

def create_classifier(cancer_type):
    # get the data
    
    # adding this path so we can import get_train_and_test
    import sys
    path = '../' # needs to be in top folder (where get_train_and_test.py is)
    sys.path.append(path)
    from get_train_and_test import get_train_and_test
    root_path = path
    
    seed = 42 # using a seed for splitting up the train and test data 
    m_values_train, m_values_test, diagnoses_train, diagnoses_test = get_train_and_test(cancer_type, use_small=False, root_path = root_path, model_path = root_path + '/simple_methods/', model_type = classifier_type + '_' + cancer_type, seed = seed)
    
    
    # fit the model
    if classifier_type == 'LR':
        from sklearn.linear_model import LogisticRegression
        model = LogisticRegression(penalty = params['penalty'], solver = params['solver'], max_iter = params['max_iter']) # sag is supposed to be fast on large datasets (if features are around same scale)
    
    elif classifier_type == 'SVM':
        from sklearn.svm import SVC
        print("params are ", params_SVM)
        model = SVC(kernel = params_SVM['kernel'], probability  = params_SVM['probability'], tol = params_SVM['tol'], gamma = params_SVM['gamma'])
    else:
        print("classifier type ", classifier_type, " not known!")
    
    
    fitted = model.fit(X=m_values_train, y=diagnoses_train)
    
    # save
    import os
    if not ('saved_models' in os.listdir()):
        os.mkdir('saved_models')
    
    import joblib # joblib is apparently more efficient than pickle functions for model saving (see https://scikit-learn.org/stable/modules/model_persistence.html)
    joblib.dump(fitted, 'saved_models/'+classifier_type+'_model_'+cancer_type+'.pkl')
    
    # Evaluate:
    from Evaluate import print_evaluation
    predictions = print_evaluation(fitted, m_values_test, diagnoses_test, classifier_type+'_'+cancer_type, save_folder = 'figs/')
    
    # create eval curves
    import matplotlib.pyplot as plt
    from Evaluate import plot_curve
    import numpy as np
    num_classes = len(np.unique(diagnoses_test, axis=0))
    print("num classes is: ", num_classes)
    confidence = fitted.predict_proba(m_values_test)
    plot_curve('roc', diagnoses_test, confidence, num_classes, classifier_type+'_'+cancer_type, save_folder = 'figs/')
    plot_curve('precision_recall', diagnoses_test, confidence, num_classes, classifier_type+'_'+cancer_type, save_folder = 'figs/')


    

import multiprocessing as mp
pool = mp.Pool(processes=mp.cpu_count())
results = [pool.map(create_classifier, (cancer_type for cancer_type in cancer_types))]



Getting values and diagnoses from: 
Getting values and diagnoses from: 
../data_preprocessing/dataset/pandas/m_values/TCGA-BLCA.csv
../data_preprocessing/dataset/pandas/m_values/TCGA-BRCA.csv
../data_preprocessing/dataset/pandas/diagnoses/TCGA-BLCA.csv
../data_preprocessing/dataset/pandas/diagnoses/TCGA-BRCA.csv
m_value and diagnoses shapes:
(278118, 436)
(436,)
m values train, m values test, diagnoses train, diagnoses test shapes:
(327, 278118) (109, 278118) (327,) (109,)
params are  {'kernel': 'rbf', 'tol': 0.001, 'gamma': 'scale', 'probability': True}
m_value and diagnoses shapes:
(277551, 889)
(889,)
m values train, m values test, diagnoses train, diagnoses test shapes:
(666, 277551) (223, 277551) (666,) (223,)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

/home/in268/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/in268/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/in268/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Accuracy: 
0.9541284403669725
Conf mat:
[[  0   5]
 [  0 104]]
Precision:  [0.         0.95412844]
Recall:  [0. 1.]
F1:  [0.         0.97652582]
Matthews Correlation Coeficient:  0.0
roc_auc for each class:  [1. 1.]
        name                     values
0   Accuracy                   0.954128
1  Precision  [0.0, 0.9541284403669725]
2     Recall                 [0.0, 1.0]
3         f1  [0.0, 0.9765258215962441]
4        mcc                          0
5    roc_auc                 [1.0, 1.0]
num classes is:  2
     Normal          
0.0     0.0       NaN
0.0     0.2       NaN
0.0     1.0       NaN
1.0     1.0       NaN
0.0     NaN  0.000000
0.0     NaN  0.009615
0.0     NaN  1.000000
1.0     NaN  1.000000
          Normal     
1.000000     1.0  NaN
0.800000     1.0  NaN
0.600000     1.0  NaN
0.400000     1.0  NaN
0.200000     1.0  NaN
...          ...  ...
0.038462     NaN  1.0
0.028846     NaN  1.0
0.019231     NaN  1.0
0.009615     NaN  1.0
0.000000     NaN  1.0

[111 rows x 2 columns]